In [1]:

!pip install git+https://github.com/huggingface/transformers.git
#!pip install git+git://github.com/AndLen/simpletransformers.git --quiet
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ezbml0dr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-ezbml0dr
  Resolved https://github.com/huggingface/transformers.git to commit 614e191c4d1c19163fe2a3b98fb78efc94661b3f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import csv
import os
import torch
from transformers import pipeline
import gc
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.special import softmax
from simpletransformers.classification import (ClassificationModel, ClassificationArgs)
import sklearn
from sklearn.model_selection import train_test_split

In [3]:
xls = pd.ExcelFile("/content/drive/MyDrive/Tweets_data/training-Obama-Romney-tweets.xlsx")
df1 = pd.read_excel(xls, 'Obama')
df2 = pd.read_excel(xls, 'Romney')


def preprocess(df):

  df = df[["Anootated tweet","Unnamed: 4"]]
  df.drop(0, axis = 0, inplace = True)
  df.rename(columns={"Anootated tweet":"text", "Unnamed: 4": "labels"}, inplace = True)
  df = df[df["labels"].isin([0,1,-1])]
  df["labels"].replace(-1,2,inplace=True)
  df = df[df["text"].notnull()]
  df.dropna(inplace=True)
  return df


In [4]:
df1 = preprocess(df1)
df2 = preprocess(df2)
df = pd.concat( [df1,df2], ignore_index = True)

<ipython-input-3-fc922de87eff>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(0, axis = 0, inplace = True)
<ipython-input-3-fc922de87eff>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"Anootated tweet":"text", "Unnamed: 4": "labels"}, inplace = True)
<ipython-input-3-fc922de87eff>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(0, axis = 0, inplace = True)
<ipython-input-3-fc922de87eff>:10: Set

In [5]:
#train_df, test_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df["labels"])
train_df = df

In [6]:
train_df = train_df.reset_index(drop=True)
#test_df = test_df.reset_index(drop=True)
train_df.head()

,text,labels
0,"Kirkpatrick, who wore a baseball cap embroider...",0
1,#<e>obama</e> debates that Cracker Ass Cracker...,1
2,@Hollivan @hereistheanswer Youre missing the ...,0
3,I was raised as a Democrat left the party yea...,2
4,The <e>Obama camp</e> can't afford to lower ex...,0


In [7]:
# This cleans ram and vram during re-runs
gc.collect()
torch.cuda.empty_cache()

In [19]:
# Create a ClassificationModel
model_args = ClassificationArgs(num_train_epochs=2, 
                                overwrite_output_dir=True)
model_args.manual_seed = 42
model_args.best_model_dir = "/content/drive/MyDrive/Tweets_data/Bertweetbest_model"
model_args.output_dir = "/content/drive/MyDrive/Tweets_data/Bertweet/output"
model_args.normalization = True #this enables the built-in Bertweet custom tokenizer

model_args.reprocess_input_data = True

model_args.train_batch_size = 80
model_args.eval_batch_size = 80

model_args.early_stopping_metric = "mcc"
model_args.early_stopping_metric_minimize = False
model_args.use_early_stopping = True
model_args.early_stopping_consider_epochs = True
model_args.early_stopping_patience = 1

model = ClassificationModel(model_type='bertweet', 
                            model_name='vinai/bertweet-base', 
                            args = model_args, 
                            num_labels = 3,
                            use_cuda=True)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

In [20]:
shuffled_training = df.sample(frac=1).reset_index(drop=True)
model.train_model(train_df,
                  acc=sklearn.metrics.accuracy_score, 
                  f1=sklearn.metrics.f1_score)

  0%|          | 0/11118 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/139 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/139 [00:00<?, ?it/s]

(278, 0.7242260241894414)

In [10]:
#result, model_output, wrong_prediction = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)


# Test Data


In [21]:
xls1 = pd.ExcelFile("/content/drive/MyDrive/Tweets_data/final-testData-no-label-Obama-tweets.xlsx")
test1 = pd.read_excel(xls1, header = None)
test1.rename(columns={0:"id", 1: "text"}, inplace = True)
xls2 = pd.ExcelFile("/content/drive/MyDrive/Tweets_data/final-testData-no-label-Romney-tweets.xlsx")
test2 = pd.read_excel(xls2, header = None)
test2.rename(columns={0:"id", 1: "text"}, inplace = True)


In [22]:
test1.head()

,id,text
0,1,<e>Obama</e> has to maintain his professionali...
1,2,<e>Obama</e> went into the debate swinging and...
2,3,Ditto. I started @247LS 4 years ago. RT @bmorr...
3,4,I absolutely love <e>Obama</e>'s view in <a>im...
4,5,I'm agreeing completely with <e>Obama</e>'s st...


In [23]:
prediction_obama, raw_outputs = model.predict(test1["text"].to_list())

  0%|          | 0/1951 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [24]:
preds_obama = pd.DataFrame(test1[["id"]])
preds_obama["output"] = prediction_obama
preds_obama["output"].replace(2,-1,inplace=True)

In [25]:
preds_obama.head()

,id,output
0,1,1
1,2,1
2,3,0
3,4,1
4,5,1


In [26]:
with open('obama.txt', 'w') as f:
  for i in range(len(preds_obama)):

    op_line = str(preds_obama["id"][i])+";;"+str(preds_obama["output"][i])
    f.write(op_line)
    f.write("\n")

In [27]:
predictions, raw_outputs = model.predict(test2["text"].to_list())
preds_romney = pd.DataFrame(test2[["id"]])
preds_romney["output"] = predictions
preds_romney["output"].replace(2,-1,inplace=True)

  0%|          | 0/1900 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [29]:
with open('romney.txt', 'w') as f:
  for i in range(len(preds_romney)):

    op_line = str(preds_romney["id"][i])+";;"+str(preds_romney["output"][i])
    f.write(op_line)
    f.write("\n")